In [ ]:
#import types
import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
import corner
import emcee
from isochrones.mist import MIST_Isochrone
from isochrones import StarModel
#from scipy.special import erfinv
from scipy.stats import norm, pearsonr

In [ ]:
chain = np.load('mcmc_out/5095_chains_1.npy')
mA_mu = chain[:,5000:,11].mean()
mA_sigma = chain[:,5000:,11].std()
mB_mu = chain[:,5000:,12].mean()
mB_sigma = chain[:,5000:,12].std()
rho, _ = pearsonr(chain[:,5000:,11].flatten(), chain[:,5000:,12].flatten())
print(mA_mu, mA_sigma)
print(mB_mu, mB_sigma)
print(rho)

plt.hist(chain[...,11].flatten(), range=[0.8, 1.4], bins=30, density=True, alpha=0.3)
plt.plot(np.linspace(0.8, 1.4, 100), norm.pdf(np.linspace(0.8, 1.4, 100), chain[...,11].mean(), chain[...,11].std()))

plt.hist(chain[...,12].flatten(), range=[0.8, 1.4], bins=30, density=True, alpha=0.3)
plt.plot(np.linspace(0.8, 1.4, 100), norm.pdf(np.linspace(0.8, 1.4, 100), chain[...,12].mean(), chain[...,12].std()))

In [ ]:
plt.hist((chain[:,5000:,12]/chain[:,5000:,11]).flatten(), bins=30);

In [ ]:
plt.scatter(chain[:,5000:,11].flatten(), chain[:,5000:,12].flatten(), marker='.', s=0.1)

In [ ]:
mist = MIST_Isochrone()

In [ ]:
mags = {'G':(13.4394, 0.0002), 'BP':(13.7217, 1/582.46), 'RP':(12.9942, 1/1167.71), 
        'J':(12.499, 0.023), 'H':(12.217, 0.018), 'K':(12.215, 0.024)}
mod = StarModel(mist, **mags, N=2, parallax=(0.8126,0.0148))
mod.print_ascii()
#mod.lnlike([1.1, 1.1,8.0, 0., 1260, 0.1])

In [ ]:
def mass_lnprior(p):
    return -1/(2*(1 - rho**2)) * ((p[0] - mA_mu)**2/mA_sigma**2 + (p[1] - mB_mu)**2/mB_sigma**2 
                                  - 2*rho*(p[0] - mA_mu)*(p[1] - mB_mu)/(mA_sigma*mB_sigma))

def lnpost(p):
    return mass_lnprior(p) + mod.lnprior(p) + mod.lnlike(p)

In [ ]:
try:
    chain = np.load('isochrones_out/isochrones_chain.npy')
    lnprobability = np.load('isochrones_out/isochrones_lnprob.npy')
except:
    from multiprocessing import Pool
    with Pool() as pool:
        nwalkers = 50
        sampler = emcee.EnsembleSampler(nwalkers, 6, lnpost, pool=pool)
        p0 = np.array([mA_mu, mB_mu, 9.6, 0.0, 1230, 0.3]) \
            + np.array([mA_sigma, mB_sigma, 0.2, 0.05, 10, 0.05])*np.random.normal(size=(nwalkers,6))
        sampler.run_mcmc(p0, 100000);
    chain = sampler.get_chain()
    np.save('isochrones_out/isochrones_chain.npy', chain)
    lnprobability = sampler.lnprobability
    np.save('isochrones_out/isochrones_lnprob.npy', lnprobability)

In [ ]:
good_walkers = lnprobability[:,-1] > -25

In [ ]:
plt.plot(chain[:,good_walkers,4]);

In [ ]:
#corner.corner(chain[20000:,good_walkers,:].reshape(-1, 6))

In [ ]:
chain[20000:,good_walkers,:].reshape(-1, 6).shape

In [ ]:
r1 = mist.radius(chain[-1,good_walkers,0], chain[-1,good_walkers,2], chain[-1,good_walkers,3])
r1.mean(), r1.std()

In [ ]:
r1 = mist.radius(chain[-1,good_walkers,1], chain[-1,good_walkers,2], chain[-1,good_walkers,3])
r1.mean(), r1.std()

In [ ]:
i,j = np.unravel_index(lnprobability.argmax(), lnprobability.shape)
p0_best = chain[j,i,:]
print(p0_best)
print(lnprobability[i,j])

In [ ]:
# from isochrones import get_ichrone, StarModel

# mags = {'G':(13.4394, 0.0002), 'BP':(13.7217, 1/582.46), 'RP':(12.9942, 1/1167.71), 
#         'J':(12.499, 0.023), 'H':(12.217, 0.018), 'K':(12.215, 0.024)}
# mod = StarModel(mist, **mags, N=1, parallax=(0.8126*np.sqrt(2),0.0148))
# mod.set_bounds(mass=(1.0, 1.2), feh=(-0.3, 0.1))
# mod.print_ascii()

In [ ]:
#mod.fit(n_live_points=2000, resume=False)

In [ ]:
#mod.corner_physical();

In [ ]:
# from isochrones import get_ichrone, StarModel

# mags = {'G':(13.4394, 0.0002), 'BP':(13.7217, 1/582.46), 'RP':(12.9942, 1/1167.71), 
#         'J':(12.499, 0.023), 'H':(12.217, 0.018), 'K':(12.215, 0.024)}
# mod = StarModel(mist, **mags, N=2, parallax=(0.8126,0.0148))
# mod.print_ascii()

In [ ]:
# import inspect
# print(inspect.getsource(mod.set_bounds))
# mod.set_bounds(mass=(1.0, 1.2), feh=(-0.3, 0.1))

In [ ]:
#mod.fit(n_live_points=2000)

In [ ]:
#mod.corner_physical();

In [ ]:
# mags = {'G':(13.4394, 0.0002), 'BP':(13.7217, 1/582.46), 'RP':(12.9942, 1/1167.71), 
#         'J':(12.499, 0.023), 'H':(12.217, 0.018), 'K':(12.215, 0.024)}
# mod = StarModel(mist, **mags, N=2, parallax=(0.8126,0.0148))
# mod.print_ascii()

In [ ]:
# def mnest_prior_modified(self, cube, ndim, nparams):
#         for _,n in self.obs.Nstars.items():
#             #minmass, maxmass = mod.bounds('mass')
#             #for j in range(n):
#             #    cube[j] = (maxmass - minmass)*cube[j] + minmass
#             #cube[0] = np.percentile(chain[...,11], 100*cube[0])
#             #cube[1] = np.percentile(chain[...,12], 100*cube[1])
#             cube[0] = mA_mu + mA_sigma*np.sqrt(2)*erfinv(2*cube[0] - 1)
#             #cube[1] = mB_mu + mB_sigma*np.sqrt(2)*erfinv(2*cube[1] - 1)
#             cube[1] = mB_mu + rho*mB_sigma/mA_sigma * (cube[0] - mA_mu) + \
#                         np.sqrt(2)*mB_sigma*np.sqrt(1 - rho**2) * erfinv(2*cube[1] - 1)

#             for j, par in enumerate(['age','feh','distance','AV']):
#                 lo, hi = self.bounds(par)
#                 cube[n+j] = (hi - lo)*cube[n+j] + lo

# mod.mnest_prior = types.MethodType(mnest_prior_modified, mod)

In [ ]:
# import logging
# def lnprior_modified(self, p):
#     N = self.obs.Nstars
#     i = 0
#     lnp = 0
#     for s in self.obs.systems:
#         age, feh, dist, AV = p[i+N[s]:i+N[s]+4]
#         for prop, val in zip(['age','feh','distance','AV'],
#                              [age, feh, dist, AV]):
#             lo,hi = self.bounds(prop)
#             if val < lo or val > hi:
#                 return -np.inf
#             lnp += np.log(self.prior(prop, val))
#             if not np.isfinite(lnp):
#                 logging.debug('lnp=-inf for {}={} (system {})'.format(prop,val,s))
#                 return -np.inf

#         masses = p[i:i+N[s]]

#         # Mass prior for primary
#         #lnp += np.log(self.prior('mass', masses[0]))
#         lnp += -1/(2*(1 - rho**2)) * ((masses[0] - mA_mu)**2/mA_sigma**2 + (masses[1] - mB_mu)**2/mB_sigma**2 -\
#                                       2*rho*(masses[0] - mA_mu)*(masses[1] - mB_mu)/(mA_sigma*mB_sigma))
#         if not np.isfinite(lnp):
#             logging.debug('lnp=-inf for mass={} (system {})'.format(masses[0],s))

#         i += N[s] + 4

#     return lnp

# mod.lnprior = types.MethodType(lnprior_modified, mod)